In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
import openpyxl
from openpyxl.drawing.image import Image
from openpyxl import Workbook
import seaborn as sns
import xlsxwriter
from io import BytesIO
import datetime as dt
import os

In [ ]:
data = pd.read_excel('Weekly Compliance_CPR 00-00-Item.xlsx')
LLE = pd.read_excel('LLE.xlsx')
SAP =pd.read_csv('SAP_DATA_LOOKUP_202408271113.csv')
df_cleaned = data
active_SAP = SAP[SAP['TEXT_EMP_STATUS'] == 'Active']

In [ ]:
columns_to_strip = ['LLE', 'E1', 'E2', 'E3', 'E4']
# Stripping trailing and leading spaces from the specified columns
LLE[columns_to_strip] = LLE[columns_to_strip].apply(lambda col: col.str.strip())

active_SAP['Full Name'] = active_SAP.apply(
    lambda row: f"{row['LAST_NAME']}, {row['FIRST_NAME']}" + 
                (f" {row['MIDDLE_NAME'][0]}" if pd.notna(row['MIDDLE_NAME']) else ""),
    axis=1
).str.strip()  # Removes any trailing spaces


In [ ]:
df_cleaned['Perner'] = df_cleaned['Perner'].astype(float).astype(int)
LLE['CUST_PERNER'] = LLE['CUST_PERNER'].astype(float).astype(int)
SAP['PERNR'] = SAP['PERNR'].astype(float).astype(int)

In [ ]:
merged_1st_step = df_cleaned.merge(LLE, left_on='Perner', right_on='CUST_PERNER', how='left')

In [ ]:
merged_2nd_step = merged_1st_step.merge(
    active_SAP[['INTERNET_MAIL_ADDRESS02', 'Full Name']].rename(
        columns={'INTERNET_MAIL_ADDRESS02': 'LLE_EMAIL', 'Full Name': 'LLE_FULL_NAME'}
    ),
    left_on='LLE', 
    right_on='LLE_FULL_NAME', 
    how='left'
)

In [ ]:
merged_final = merged_2nd_step.merge(
    active_SAP[['INTERNET_MAIL_ADDRESS02', 'Full Name']].rename(
        columns={'INTERNET_MAIL_ADDRESS02': 'E1_EMAIL', 'Full Name': 'E2_FULL_NAME'}
    ),
    left_on='E2', 
    right_on='E2_FULL_NAME', 
    how='left'
)

In [ ]:
#merged_final['Required By Date'] = pd.to_datetime(merged_final['Required By Date'])
#merged_final['Completion Date'] = pd.to_datetime(merged_final['Completion Date'])

In [ ]:
def create_bar_chart(data, title, color_palette):
    """Generate a bar chart using seaborn."""
    plt.figure(figsize=(8, 6))
    sns.set_theme(style="whitegrid")
    ax = sns.barplot(x=data.index, y=data.values, palette=color_palette)
    ax.set_title(title, fontsize=14)
    ax.set_ylabel('Count')
    ax.set_xlabel('Learner Item Status')
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    # Save the plot to a BytesIO object
    chart_stream = BytesIO()
    plt.savefig(chart_stream, format='png')
    plt.close()
    chart_stream.seek(0)
    return chart_stream

def create_pie_chart(data, title, color_palette):
    """Generate a pie chart."""
    plt.figure(figsize=(6, 6))
    plt.pie(data.values, labels=data.index, autopct='%1.1f%%', colors=color_palette)
    plt.title(title, fontsize=14)
    plt.tight_layout()
    
    # Save the plot to a BytesIO object
    chart_stream = BytesIO()
    plt.savefig(chart_stream, format='png')
    plt.close()
    chart_stream.seek(0)
    return chart_stream

def format_excel_sheet(workbook, worksheet, df, start_row=0, start_col=0):
    """Format an Excel sheet with pandas DataFrame, handling NaT and NaN."""
    header_format = workbook.add_format({'bold': True, 'bg_color': '#FFC0CB', 'border': 1})
    cell_format = workbook.add_format({'border': 1})
    
    # Convert all datetime columns to strings
    for col in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[col]):
            df[col] = df[col].astype(str).replace('NaT', '')



In [ ]:
def sanitize_sheet_name(name):
    """Truncate the sheet name to be 31 characters or fewer."""
    return name[:31]

In [ ]:
def create_combined_chart(bar_data, pie_data, title, bar_color_palette, pie_color_palette, breakdown_table):
    """Create a combined chart with a bar plot, pie chart, and breakdown table."""
    fig, ax = plt.subplots(1, 2, figsize=(14, 7))
    
    # Bar Chart
    sns.barplot(x=bar_data.index, y=bar_data.values, palette=bar_color_palette, ax=ax[0])
    ax[0].set_title(title, fontsize=16)
    ax[0].set_ylabel('Count')
    ax[0].set_xlabel('Learner Item Status')
    ax[0].set_ylim(0, max(bar_data.values) * 1.2)  # Normalize the bar height
    for i, v in enumerate(bar_data.values):
        ax[0].text(i, v + 0.05 * max(bar_data.values), str(v), ha='center')

    # Pie Chart
    ax[1].pie(pie_data.values, labels=pie_data.index, autopct='%1.1f%%', colors=pie_color_palette)
    ax[1].set_title('Percentage Breakdown', fontsize=16)

    # Add Breakdown Table using tabulate
    table_str = tabulate(breakdown_table, headers='keys', tablefmt='grid', showindex=False)
    fig.text(0.5, -0.25, table_str, ha='center', fontsize=12, wrap=True)

    # Save the combined chart to a BytesIO object
    chart_stream = BytesIO()
    plt.savefig(chart_stream, format='png', bbox_inches='tight')
    plt.close()
    chart_stream.seek(0)
    return chart_stream

def create_org_breakdown_chart(data, title, color_palette):
    """Create a bar chart for Org Breakdown with LLE on the x-axis and status breakdown."""
    fig, ax = plt.subplots(figsize=(12, 8))
    
    data.plot(kind='bar', stacked=True, color=color_palette, ax=ax)
    ax.set_title(title, fontsize=16)
    ax.set_ylabel('Count')
    ax.set_xlabel('LLE')
    ax.legend(title='Learner Item Status', bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Save the org breakdown chart to a BytesIO object
    chart_stream = BytesIO()
    plt.savefig(chart_stream, format='png', bbox_inches='tight')
    plt.close()
    chart_stream.seek(0)
    return chart_stream

def generate_reports(merged_final):
    # List of unique E2 executives
    executives = merged_final['E2_FULL_NAME'].unique()
    
    # Define color palettes
    ba_color_palette = ['#0072BB', '#FFD700', '#FF6347']  # Example Disney-themed colors: blue, gold, tomato
    e2_color_palette = ['#102336', '#5D88BB', '#B3CBE4']  # Blue gradient

    # Define the directory where the reports will be saved
    save_directory = r'J:\Data\RMSA Analysis\0 - Team Working Files\10 - Nick'
    
    # Ensure the directory exists
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
    
    # Iterate over each executive to create their specific report
    for exec_name in executives:
        exec_data = merged_final[merged_final['E2_FULL_NAME'] == exec_name]
        
        # Skip if exec_data is empty
        if exec_data.empty:
            print(f"No data found for executive: {exec_name}. Skipping report generation.")
            continue
        
        ba_text = exec_data['BA Text'].iloc[0]  # Assuming each exec belongs to one BA
        
        # Define the path to save the Excel file
        save_path = os.path.join(save_directory, f'{exec_name}_Report.xlsx')
        
        # Create an Excel file for each executive
        with pd.ExcelWriter(save_path, engine='xlsxwriter') as writer:
            workbook = writer.book
            
            # Tab 1: BA Overview
            ba_data = merged_final[merged_final['BA Text'] == ba_text]
            ba_summary = ba_data['Learner Item Status'].value_counts()
            ba_pie_data = ba_summary / ba_summary.sum()
            ba_breakdown_table = pd.DataFrame({
                'Learner Item Status': ba_summary.index,
                'Count': ba_summary.values,
                'Percentage': (ba_summary.values / ba_summary.sum() * 100).round(1)
            })
            ba_chart = create_combined_chart(ba_summary, ba_pie_data, f'{ba_text} Overview', ba_color_palette, ba_color_palette, ba_breakdown_table)
            
            # Create the BA Overview tab
            sheet_name = sanitize_sheet_name(f'{ba_text} Overview')
            worksheet = workbook.add_worksheet(sheet_name)
            worksheet.insert_image('A1', '', {'image_data': ba_chart})
            
            # Tab 2: Executive-specific overview
            exec_summary = exec_data['Learner Item Status'].value_counts()
            exec_pie_data = exec_summary / exec_summary.sum()
            exec_breakdown_table = pd.DataFrame({
                'Learner Item Status': exec_summary.index,
                'Count': exec_summary.values,
                'Percentage': (exec_summary.values / exec_summary.sum() * 100).round(1)
            })
            exec_chart = create_combined_chart(exec_summary, exec_pie_data, f'{exec_name} Overview', e2_color_palette, e2_color_palette, exec_breakdown_table)
            
            # Create the Executive Overview tab
            exec_overview_tab = sanitize_sheet_name(f'{exec_name} Overview')
            worksheet = workbook.add_worksheet(exec_overview_tab)
            worksheet.insert_image('A1', '', {'image_data': exec_chart})
            
            # Tab 3: Org Breakdown
            lle_summary = exec_data.groupby(['LLE', 'Learner Item Status']).size().unstack(fill_value=0)
            lle_chart = create_org_breakdown_chart(lle_summary, f'{exec_name} Org Breakdown', e2_color_palette)
            
            sheet_name = sanitize_sheet_name('Org Breakdown')
            worksheet = workbook.add_worksheet(sheet_name)
            worksheet.insert_image('A1', '', {'image_data': lle_chart})
            format_excel_sheet(workbook, worksheet, lle_summary.reset_index(), start_row=20)
            
            # Tab 4: Org_Overdue
            overdue_data = exec_data[exec_data['Learner Item Status'] == 'Overdue']
            if not overdue_data.empty:
                sheet_name = sanitize_sheet_name('Org Overdue')
                overdue_data.to_excel(writer, sheet_name=sheet_name, index=False)
            
            # Tab 5: Org_Coming_Due
            coming_due_data = exec_data[exec_data['Learner Item Status'] == 'Coming Due']
            if not coming_due_data.empty:
                sheet_name = sanitize_sheet_name('Org Coming Due')
                coming_due_data.to_excel(writer, sheet_name=sheet_name, index=False)
            
            # Tab 6: Org_Current
            current_data = exec_data[exec_data['Learner Item Status'] == 'Current']
            if not current_data.empty:
                sheet_name = sanitize_sheet_name('Org Current')
                current_data.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"Reports generated successfully and saved to {save_directory}.")

In [ ]:
generate_reports(merged_final)

In [12]:
def create_combined_chart(bar_data, pie_data, title, bar_color_palette, pie_color_palette, breakdown_table):
    """Create a combined chart with a bar plot, pie chart, and breakdown table."""
    fig, ax = plt.subplots(1, 2, figsize=(14, 7))
    
    # Bar Chart
    sns.barplot(x=bar_data.index, y=bar_data.values, palette=bar_color_palette, ax=ax[0])
    ax[0].set_title(title, fontsize=16)
    ax[0].set_ylabel('Count')
    ax[0].set_xlabel('Learner Item Status')
    ax[0].set_ylim(0, max(bar_data.values) * 1.2)  # Normalize the bar height
    for i, v in enumerate(bar_data.values):
        ax[0].text(i, v + 0.05 * max(bar_data.values), str(v), ha='center')

    # Pie Chart
    ax[1].pie(pie_data.values, labels=pie_data.index, autopct='%1.1f%%', colors=pie_color_palette)
    ax[1].set_title('Percentage Breakdown', fontsize=16)

    # Add Breakdown Table
    fig.text(0.5, -0.1, breakdown_table.to_string(index=False), ha='center', fontsize=12, wrap=True)

    # Save the combined chart to a BytesIO object
    chart_stream = BytesIO()
    plt.savefig(chart_stream, format='png', bbox_inches='tight')
    plt.close()
    chart_stream.seek(0)
    return chart_stream

def generate_reports(merged_final):
    # List of unique E2 executives
    executives = merged_final['E2_FULL_NAME'].unique()
    
    # Define color palettes
    ba_color_palette = ['#0072BB', '#FFD700', '#FF6347']  # Example Disney-themed colors: blue, gold, tomato
    e2_color_palette = ['#102336', '#5D88BB', '#B3CBE4']  # Blue gradient

    # Define the directory where the reports will be saved
    save_directory = r'J:\Data\RMSA Analysis\0 - Team Working Files\10 - Nick'
    
    # Ensure the directory exists
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
    
    # Iterate over each executive to create their specific report
    for exec_name in executives:
        exec_data = merged_final[merged_final['E2_FULL_NAME'] == exec_name]
        
        # Skip if exec_data is empty
        if exec_data.empty:
            print(f"No data found for executive: {exec_name}. Skipping report generation.")
            continue
        
        ba_text = exec_data['BA Text'].iloc[0]  # Assuming each exec belongs to one BA
        
        # Define the path to save the Excel file
        save_path = os.path.join(save_directory, f'{exec_name}_Report.xlsx')
        
        # Create an Excel file for each executive
        with pd.ExcelWriter(save_path, engine='xlsxwriter') as writer:
            workbook = writer.book
            
            # Tab 1: BA Overview
            ba_data = merged_final[merged_final['BA Text'] == ba_text]
            ba_summary = ba_data['Learner Item Status'].value_counts()
            ba_pie_data = ba_summary / ba_summary.sum()
            ba_breakdown_table = pd.DataFrame({
                'Learner Item Status': ba_summary.index,
                'Count': ba_summary.values,
                'Percentage': (ba_summary.values / ba_summary.sum()) * 100
            })
            ba_chart = create_combined_chart(ba_summary, ba_pie_data, f'{ba_text} Overview', ba_color_palette, ba_color_palette, ba_breakdown_table)
            
            # Create the BA Overview tab
            sheet_name = sanitize_sheet_name(f'{ba_text} Overview')
            worksheet = workbook.add_worksheet(sheet_name)
            worksheet.insert_image('A1', '', {'image_data': ba_chart})
            
            # Tab 2: Executive-specific overview
            exec_summary = exec_data['Learner Item Status'].value_counts()
            exec_pie_data = exec_summary / exec_summary.sum()
            exec_breakdown_table = pd.DataFrame({
                'Learner Item Status': exec_summary.index,
                'Count': exec_summary.values,
                'Percentage': (exec_summary.values / exec_summary.sum()) * 100
            })
            exec_chart = create_combined_chart(exec_summary, exec_pie_data, f'{exec_name} Overview', e2_color_palette, e2_color_palette, exec_breakdown_table)
            
            # Create the Executive Overview tab
            exec_overview_tab = sanitize_sheet_name(f'{exec_name} Overview')
            worksheet = workbook.add_worksheet(exec_overview_tab)
            worksheet.insert_image('A1', '', {'image_data': exec_chart})
            
            # Tab 3: Org Breakdown
            lle_summary = exec_data.groupby('LLE')['Learner Item Status'].value_counts().unstack(fill_value=0)
            lle_chart = create_combined_chart(lle_summary.sum(axis=1), lle_summary.sum(axis=1) / lle_summary.sum(axis=1).sum(), f'{exec_name} Org Breakdown', e2_color_palette, e2_color_palette, lle_summary.reset_index())
            
            sheet_name = sanitize_sheet_name('Org Breakdown')
            worksheet = workbook.add_worksheet(sheet_name)
            worksheet.insert_image('A1', '', {'image_data': lle_chart})
            
            # Tab 4: Org_Overdue
            overdue_data = exec_data[exec_data['Learner Item Status'] == 'Overdue']
            if not overdue_data.empty:
                sheet_name = sanitize_sheet_name('Org Overdue')
                overdue_data.to_excel(writer, sheet_name=sheet_name, index=False)
            
            # Tab 5: Org_Coming_Due
            coming_due_data = exec_data[exec_data['Learner Item Status'] == 'Coming Due']
            if not coming_due_data.empty:
                sheet_name = sanitize_sheet_name('Org Coming Due')
                coming_due_data.to_excel(writer, sheet_name=sheet_name, index=False)
            
            # Tab 6: Org_Current
            current_data = exec_data[exec_data['Learner Item Status'] == 'Current']
            if not current_data.empty:
                sheet_name = sanitize_sheet_name('Org Current')
                current_data.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"Reports generated successfully and saved to {save_directory}.")


In [ ]:
generate_reports(merged_final)